In [1]:
### Notes on 7D35
# Human LC8 bound to ebola virus VP35(67-76)

In [26]:
#Imports
import MDAnalysis as mda
from MDAnalysis import Universe
import MDAnalysis.tests.datafiles
from MDAnalysis.tests.datafiles import PSF, DCD, GRO, XTC
import MDAnalysis.analysis.base as AnalysisBase
from MDAnalysis.analysis import align, rms

import matplotlib.pyplot as plt
import numpy as np

In [22]:
#loading various file types
pdb = mda.Universe('step3_input.pdb', permissive=True)
gro = mda.Universe('step3_input.gro', permissive=True)
xtc = mda.Universe('7D35_md_100ns.xtc', permissive=True)

#topology and traj files to create universe 
universe = mda.Universe('7D35_md_100ns.tpr', '7D35_md_100ns.xtc')
#simplify
u = universe

In [20]:
print(len(u.trajectory))

11


In [28]:
ref = mda.Universe('step3_input.pdb', permissive=True)
mobile = mda.Universe('7D35_md_100ns.tpr', '7D35_md_100ns.xtc')
mobile_CA = mobile.select_atoms("name CA")
ref_CA = ref.select_atoms("name CA")
MDAnalysis.analysis.rms.rmsd(mobile_CA.positions, ref_CA.positions)

0.36834528266730576

In [10]:
### Analysis 
#https://docs.mdanalysis.org/1.1.1/documentation_pages/analysis_modules.html#building-blocks-for-analysis

In [11]:
### Analysis building blocks
# class MDAnalysis.analysis.base.AnalysisBase

class NewAnalysis():
    def __init__(self, atomgroup):
        super(NewAnalysis, self).__init__(atomgroup.universe.trajectory)
        self._parameter = parameter
        self._ag = atomgroup

    def _prepare(self):
        # OPTIONAL
        # Called before iteration on the trajectory has begun.
        # Data structures can be set up at this time
        self.result = []

    def _single_frame(self):
        # REQUIRED
        # Called after the trajectory is moved onto each new frame.
        # store result of `some_function` for a single frame
        self.result.append(some_function(self._ag, self._parameter))

    def _conclude(self):
        # OPTIONAL
        # Called once iteration on the trajectory is finished.
        # Apply normalisation and averaging to results here.
        self.result = np.asarray(self.result) / np.sum(self.result)

In [12]:
####
def rotation_matrix(mobile, ref):
    return mda.analysis.align.rotation_matrix(mobile, ref)[0]

In [19]:
RotationMatrix = AnalysisBase.analysis_class(rotation_matrix)
rot = RotationMatrix(u.trajectory, mobile, ref).run(step=2)
print(rot.results)

NameError: name 'mobile' is not defined

In [ ]:
Rgyr = []
protein = u.select_atoms("protein")
for ts in u.trajectory:
    Rgyr.append((u.trajectory.time, protein.radius_of_gyration()))
Rgyr = np.array(Rgyr)

x = Rgyr[:,0]
y = Rgyr[:,1]

fig, axs = plt.subplots(1, figsize=(10,10))
axs.plot(Rgyr[:,0], Rgyr[:,1], 'b--', lw=2, label=r"$R_G$")
axs.plot()
axs.set_xlabel("time (ps)")
axs.set_ylabel(r"radius of gyration $R_G$ ($\AA$)")

z = np.polyfit(x, y, 1)
p = np.poly1d(z)
plt.plot(x,p(x),"red")


# ax.figure.savefig("Rgyr.pdf")
plt.draw()

In [ ]:
u = universe
ref = universe

import MDAnalysis.analysis.rms

R = MDAnalysis.analysis.rms.RMSD(u, ref,
           select="backbone",             # superimpose on whole backbone of the whole protein
           groupselections=["backbone and (resid 1-29 or resid 60-121 or resid 160-214)",   # CORE
                            "backbone and resid 122-159",                                   # LID
                            "backbone and resid 30-59"])                                    # NMP
R.run()

import matplotlib.pyplot as plt
rmsd = R.rmsd.T   # transpose makes it easier for plotting
time = rmsd[1]
fig, axs = plt.subplots(1, figsize=(10,10))
axs.plot(time, rmsd[2], 'k-',  label="all")
axs.legend(loc="best")
axs.set_xlabel("time (ps)")
axs.set_ylabel(r"RMSD ($\AA$)")
